In [1]:
using JuMP, ConditionalJuMP, Gurobi

In [7]:
m = Model(solver=GurobiSolver())
@variable m -1 <= x <= 1
@variable m -1 <= y <= 1
@implies m (x <= 0) => (y == 0.5)
@constraint m y >= x
d = -1
@objective m Min ((x - d)^2 + (y - (d - 0.5))^2)
solve(m)
getvalue(x), getvalue(y)

Optimize a model with 6 rows, 3 columns and 12 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  Objective range  [2e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 3 columns, 11 nonzeros
Presolved model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)

Root relaxation: objective -5.000000e-02, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.05000    0    1          -   -0.05000      -     -    0s
H    0     0                       0.0000000   -0.05000      -     -    0s

Explored 0 nodes (6 simplex iterations) in 0.00 seconds
Thread count was 12 (of 12 avail

(0.0, 0.0)

In [71]:
m = Model(solver=GurobiSolver())
@variable m 0.5 <= x <= 1
@variable m y
@constraint m y >= x
@objective m Min x^2 + y^2
# c = @constraint m x == -0.5
solve(m)
m

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [5e-01, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 5.00000000e-01


Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Gurobi

In [72]:
getdual(x)

2.0

In [73]:
JuMP.prepConstrMatrix(m)

1×2 SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [1, 1]  =  -1.0
  [1, 2]  =  1.0

In [75]:
JuMP.prepConstrBounds(m)

([-0.0], [Inf])

In [76]:
speye(10)

10×10 SparseMatrixCSC{Float64,Int64} with 10 stored entries:
  [1 ,  1]  =  1.0
  [2 ,  2]  =  1.0
  [3 ,  3]  =  1.0
  [4 ,  4]  =  1.0
  [5 ,  5]  =  1.0
  [6 ,  6]  =  1.0
  [7 ,  7]  =  1.0
  [8 ,  8]  =  1.0
  [9 ,  9]  =  1.0
  [10, 10]  =  1.0

In [79]:
@edit JuMP.prepConstrMatrix(m)

In [77]:
function linear_constraints(m::Model) 
    A = JuMP.prepConstrMatrix(m)
    b_lower, b_upper = JuMP.prepConstrBounds(m)
    x_lower = m.colLower
    x_upper = m.colUpper
    
    A = vcat(A, speye(size(A, 2)))
    b_lower = vcat(b_lower, x_lower)
    b_upper = vcat(b_upper, x_upper)
    A, b_lower, b_upper
end


linear_constraints (generic function with 1 method)

In [100]:
.![true, true]

2-element BitArray{1}:
 false
 false

In [101]:
m.obj

x² + y²

In [ ]:
m.obj.

In [ ]:
function canonicalize(m::Model, params::AbstractArray{Variable}, eps=1e-3)
    nvars = length(m.colCat)
    A_active = SparseVector{Float64, Int64}[]
    b_active = Float64[]
    
    for i in 1:nvars
        var = Variable(m, i)
        λ = getdual(var)
        if abs(λ) > eps
            if λ > 0
                ai = sparsevec([i], [-1.0])
                bi = -m.colLower[i]
            else
                ai = sparsevec([i], [1.0])
                bi = m.colUpper[i]
            end 
            push!(A_active, ai)
            push!(b_active, bi)
        end
    end
    
    nconstr = length(m.linconstr)
    for (constraint, λ) in zip(m.linconstr, m.linconstrDuals)
        if abs(λ) > eps
            if λ > 0
                ai = -sparsevec([var.col for var in constraint.terms.vars], constraint.terms.coeffs)
                bi = -constr.lb
            else
                ai = sparsevec([var.col for var in constraint.terms.vars], constraint.terms.coeffs)
                bi = constr.ub
            end
            push!(A_active, ai)
            push!(b_active, bi)
        end
    end
    
    param_cols = Set([v.col for v in params])
    isparam = collect(1:nvars) .∈ param_cols
    Ã = vcat(A_active...)
    W̃ = vcat(b_active...)
    G̃ = Ã[:, .!isparam]
    S = .-Ã[:, isparam]
    
    perm = Permutation(vcat([i for i in 1:nvars if !(i in param_cols)], 
            
            
    
    nx = length(params)
    nz = nvars - nx
    G_active = Vector{Float64}[]
    W_active = Float64[]
    S_active = Vector{Float64}[]
    H = zeros(nz, nz)
    F = zeros(nx, nz)
    
    for i in 1:nvars
        var = Variable(m, i)
        if abs(getdual(var)) > eps
            gi = zeros(nz)
            wi = 0
            Si = zeros(nx)
            if var in params
                
            